In [1]:
!pip3 install torch==1.7.1
!pip3 install tfrecord

  Using cached https://files.pythonhosted.org/packages/90/4f/acf48b3a18a8f9223c6616647f0a011a5713a985336088d7c76f3a211374/torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/fd/247ef25f5ec5f9acecfbc98ca3c6aaf66716cf52509aca9a93583d410493/protobuf-3.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Usin

In [2]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from tfrecord.torch.dataset import TFRecordDataset
import tensorflow as tf
import math
import numpy as np
from models import *

In [3]:
path_data = "/data/jcabrera/didi_wo_text/training/"
name_file = "diagrams_wo_text_20200131-00000-of-00010"

In [4]:
tfrecord_path = path_data + name_file
index_path = None

In [5]:
description = {"key": "byte", "label_id":"byte",
               "ink":"float", "stroke_length":"int", "shape":"int", "num_strokes":"int", 
               "rdp_ink":"float", "rdp_stroke_length":"int", 
               "rdp_shape":"int", "rdp_num_strokes":"int",
               }

In [6]:
dataset = TFRecordDataset(tfrecord_path, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [7]:
dataiter = iter(loader)
data = dataiter.next()
print(data)

{'key': tensor([[ 48,  48,  51,  48,  55,  56,  57,  97,  57,  54,  97,  97, 101,  55,
         101,  97]], dtype=torch.uint8), 'label_id': tensor([[101,  50,  51,  56,  56,  97, 100,  56,  55,  98, 102,  55,  97,  52,
          98,  54,  49,  97,  51,  99,  52,  97,  55,  52, 102,  51,  49,  53,
          48, 101,  51, 100,  98,  97,  57,  56,  49, 100,  49,  97]],
       dtype=torch.uint8), 'ink': tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[ 32,  32,  25,  43, 128,  31,  33,  34,  29]], dtype=torch.int32), 'shape': tensor([[  9, 128,   4]], dtype=torch.int32), 'num_strokes': tensor([[9]], dtype=torch.int32), 'rdp_ink': tensor([[187.4000, 751.9100,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[38, 36, 33, 21, 29, 19, 16, 13, 16]], dtype=torch.int32), 'rdp_shape': tensor([[ 9, 38,  4]], dtype=torch.int32), 'rdp_num_strokes': tensor([[9]], dtype=torch.int32)}


/home/shuaman/.local/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])


In [8]:
#d model cantidad de features: 64
#
xd = data["ink"].reshape((9,128,4))[:,:,0:3]
stroke_lengths = data["stroke_length"].numpy().reshape(-1)

In [9]:
stroke_lengths.reshape(-1)[0]

32

In [10]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [11]:
encoder = Encoder(d_model=64, nhead=4, dff=128, nlayers=6, size_embedding=256, dropout = 0)

In [12]:
encoder = encoder.to(device)
xd = xd.to(device)

In [13]:
output = encoder(xd, stroke_lengths, None)

In [14]:
xd.shape

torch.Size([9, 128, 3])

In [15]:
xd

tensor([[[1.5168e-01, 5.7804e-01, 0.0000e+00],
         [1.3512e-01, 5.8202e-01, 2.5000e+01],
         [1.2253e-01, 5.8489e-01, 4.5000e+01],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.1419e+00, 8.1085e-02, 9.9100e+02],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[2.0582e+00, 6.3554e-01, 1.9720e+03],
         [2.0203e+00, 6.3852e-01, 1.9940e+03],
         [1.9913e+00, 6.4745e-01, 2.0140e+03],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        ...,

        [[1.5922e+00, 7.5728e-01, 8.1820e+03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]

In [16]:
output

tensor([[-0.8219, -0.6531, -0.1365,  ..., -0.2305,  0.0314,  0.0276],
        [-0.8219, -0.6531, -0.1367,  ..., -0.2304,  0.0315,  0.0276],
        [-0.9356, -0.4667,  0.0240,  ..., -0.4332, -0.1367,  0.3417],
        ...,
        [-0.9348, -0.5076, -0.2448,  ..., -0.2927,  0.0137,  0.1151],
        [-0.9348, -0.5075, -0.2447,  ..., -0.2927,  0.0137,  0.1149],
        [-0.8200, -0.5455, -0.1384,  ..., -0.3922, -0.0516,  0.2164]],
       device='cuda:1', grad_fn=<AddmmBackward>)

In [17]:
output.shape

torch.Size([9, 256])

TypeError: __init__() missing 2 required positional arguments: 'n_components' and 'n_features'

In [ ]:
class Decoder(nn.Module):
    def __init__(self, size_embedding):
        self.dense1 = nn.Linear(in_features=size_embedding, out_features=512)
        self.dense2 = nn.Linear(in_features=512, out_features=512)
        self.dense3 = nn.Linear(in_features=512, out_features=512)
        self.dense4 = nn.Linear(in_features=512, out_features=512)
    
    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        x = F.relu(self.dense4(x))